In [1]:
import cv2
import numpy as np
import os
import torch
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox
from utils.plots import plot_skeleton_kpts

# 1. CLAHE untuk meningkatkan kontras
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    image_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
    return image_clahe

In [2]:
# 2. Retinex Deillumination untuk koreksi pencahayaan
def retinex_deillumination(image, levels=5):
    def gaussian_pyramid(image, levels):
        pyramid = [image]
        for _ in range(levels - 1):
            image = cv2.pyrDown(image)
            pyramid.append(image)
        return pyramid

    def laplacian_pyramid(pyramid):
        laplacian = []
        for i in range(len(pyramid) - 1):
            upsampled = cv2.pyrUp(pyramid[i + 1], dstsize=(pyramid[i].shape[1], pyramid[i].shape[0]))
            laplacian.append(cv2.subtract(pyramid[i], upsampled))
        laplacian.append(pyramid[-1])
        return laplacian

    def reconstruct_image(laplacian):
        image = laplacian[-1]
        for i in range(len(laplacian) - 2, -1, -1):
            image = cv2.pyrUp(image, dstsize=(laplacian[i].shape[1], laplacian[i].shape[0]))
            image = cv2.add(image, laplacian[i])
        return image

    # Buat Gaussian pyramid
    pyramid = gaussian_pyramid(image, levels)

    # Buat Laplacian pyramid
    laplacian = laplacian_pyramid(pyramid)

    # Rekonstruksi gambar
    reconstructed = reconstruct_image(laplacian)

    return reconstructed

In [3]:
# 3. Preprocessing pipeline
def preprocess_image(image):
    # CLAHE untuk meningkatkan kontras
    image_clahe = apply_clahe(image)

    # Retinex untuk koreksi pencahayaan
    image_retinex = retinex_deillumination(image_clahe)

    return image_retinex

# 4. Load YOLOv7-W6 Pose model
def load_model(weights_path):
    model = attempt_load(weights_path, map_location=torch.device('cpu'))  # Gunakan 'cuda' jika GPU tersedia
    return model

In [4]:
# 5. Deteksi pose menggunakan YOLOv7-W6 Pose

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    # Hitung vektor ba dan bc
    ba = a - b
    bc = c - b
    
    # Hitung cosine sudut menggunakan dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)  # Dalam radian
    return np.degrees(angle)  # Konversi ke derajat
    
def detect_pose(model, image, img_size=640, fall_threshold=60):
    # Resize dan normalisasi gambar
    img = letterbox(image, img_size, stride=64, auto=True)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR ke RGB, HWC ke CHW
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).float()
    img /= 255.0  # Normalisasi ke [0, 1]
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    with torch.no_grad():
        pred = model(img)[0]
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

    # Plot hasil deteksi
    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image.shape).round()
            for *xyxy, conf, cls, kpts in det:
                # Pastikan kpts adalah tensor 2D dengan shape (N, 3)
                if kpts.dim() == 2 and kpts.shape[1] == 3:  # Format: [num_keypoints, 3]
                    # Ambil keypoints yang relevan (misalnya, kepala, pinggang, kaki)
                    head = kpts[0].cpu().numpy()  # Keypoint kepala (indeks 0)
                    waist = kpts[11].cpu().numpy()  # Keypoint pinggang (indeks 11)
                    left_foot = kpts[15].cpu().numpy()  # Keypoint kaki kiri (indeks 15)
                    right_foot = kpts[16].cpu().numpy()  # Keypoint kaki kanan (indeks 16)

                    # Hitung sudut antara kepala, pinggang, dan kaki
                    angle = calculate_angle(head[:2], waist[:2], left_foot[:2])

                    # Fall detection logic
                    if angle < fall_threshold:  # Jika sudut kurang dari threshold, deteksi sebagai fall
                        cv2.putText(image, "FALL DETECTED", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        print("Fall detected!")

                    # Plot keypoints
                    plot_skeleton_kpts(image, kpts, steps=3)
                else:
                    print(f"No valid keypoints detected in this frame. Keypoints shape: {kpts.shape}")
        else:
            print("No detections in this frame.")

    return image

In [5]:
def read_ground_truth(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    
    ground_truth = []
    for line in lines:
        line = line.strip()
        if not line:  # Skip baris kosong
            continue
        try:
            # Parsing baris: frame_num, fall_status, x, y, w, h
            frame_num, fall_status, x, y, w, h = map(int, line.split(','))
            ground_truth.append((frame_num, fall_status, x, y, w, h))
        except ValueError as e:
            print(f"Error parsing line: {line}. Skipping this line. Error: {e}")
    
    # Jika file annotation tidak memiliki frame_start dan frame_end, kita bisa menghitungnya
    if ground_truth:
        frame_start = ground_truth[0][0]  # Frame awal adalah frame pertama
        frame_end = ground_truth[-1][0]   # Frame akhir adalah frame terakhir
    else:
        frame_start, frame_end = 0, 0  # Default jika tidak ada data
    
    return frame_start, frame_end, ground_truth

In [6]:
# 7. Main pipeline untuk video

# Global flag untuk kontrol proses
stop_processing = False  # Untuk menghentikan semua proses
skip_video = False  # Untuk melewati video saat ini

def process_video(video_path, annotation_path, model):
    global stop_processing, skip_video

    # Baca ground truth
    frame_start, frame_end, ground_truth = read_ground_truth(annotation_path)

    # Buka video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        if stop_processing:  # Jika 'q' ditekan, hentikan semua
            break
        if skip_video:  # Jika 'n' ditekan, lompat ke video berikutnya
            skip_video = False  # Reset flag agar hanya skip 1 video
            break
            
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocessing frame
        processed_frame = preprocess_image(frame)

        # Deteksi pose
        result_frame = detect_pose(model, processed_frame)

        # Tampilkan ground truth bounding box (jika ada)
        if frame_count >= frame_start and frame_count <= frame_end:
            for gt in ground_truth:
                if gt[0] == frame_count:
                    x, y, w, h = gt[2], gt[3], gt[4], gt[5]
                    cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Gambar bounding box

        # Tampilkan hasil
        cv2.imshow("Result", result_frame)

        # Baca tombol yang ditekan
        key = cv2.waitKey(1) & 0xFF  
        if key == ord('q'):  # Jika 'q' ditekan, hentikan semua proses
            stop_processing = True
            break
        elif key == ord('n'):  # Jika 'n' ditekan, skip ke video berikutnya
            skip_video = True
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [7]:
# 8. Main pipeline untuk dataset
if __name__ == "__main__":
    # Load model YOLOv7-W6 Pose
    weights_path = "yolov7-w6-pose.pt"  # Ganti dengan path model Anda
    model = load_model(weights_path)

    # Path ke dataset
    dataset_path = r"C:\Users\LENOVO\Documents\A Skripsi\datasets\FallDataset\Dataset"  # Ganti dengan path dataset Anda

    # Loop melalui setiap folder di dataset
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            annotation_folder = os.path.join(folder_path, "Annotation_files")
            video_folder = os.path.join(folder_path, "Videos")

            # Periksa apakah folder Annotation_files dan Videos ada
            if not os.path.exists(annotation_folder):
                print(f"Folder Annotation_files tidak ditemukan di {folder_path}. Melanjutkan ke folder berikutnya.")
                continue
            if not os.path.exists(video_folder):
                print(f"Folder Videos tidak ditemukan di {folder_path}. Melanjutkan ke folder berikutnya.")
                continue

            # Loop melalui setiap file di folder Annotation_files
            for annotation_file in os.listdir(annotation_folder):
                if annotation_file.endswith(".txt"):
                    annotation_path = os.path.join(annotation_folder, annotation_file)
                    video_file = annotation_file.replace(".txt", ".avi")
                    video_path = os.path.join(video_folder, video_file)

                    # Periksa apakah file video ada
                    if not os.path.exists(video_path):
                        print(f"File video {video_file} tidak ditemukan di {video_folder}. Melanjutkan ke file berikutnya.")
                        continue

                    # Proses video
                    print(f"Processing {video_file}...")
                    process_video(video_path, annotation_path, model)

C:\Users\LENOVO\yolov7\models\experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(w, map_location=map_location)  # load


Fusing layers... 


C:\Users\LENOVO\anaconda3\envs\yolov7_env\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Processing video (1).avi...
Error parsing line: 48. Skipping this line. Error: not enough values to unpack (expected 6, got 1)
Error parsing line: 80. Skipping this line. Error: not enough values to unpack (expected 6, got 1)
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detec